In [82]:
import matplotlib.pyplot as plt
import numpy as np

In [83]:
def Norm(m: np.ndarray) -> np.float32:
    if len(m.shape) == 1:
        return max(m)
    else:
        mx = 0
        for i in range(m.shape[0]):
            tmp = 0
            for j in range(m.shape[0]):
                tmp += np.abs(m[i][j])
            if tmp > mx:
                mx = tmp
        return mx

In [84]:
def LU_dec(A: np.ndarray) -> tuple:
    L = np.empty(A.shape)
    U = np.empty(A.shape)
    n = A.shape[0]
    for i in range(n):
        for j in range(n):
            U[i, j] = A[i, j]
            L[i, j] = int(i == j)
    for i in range(n):
        for j in range(i+1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j] = U[j] - L[j, i] * U[i]
    return (L, U)

In [85]:
def swap(A: np.ndarray, L: np.ndarray, permutations: list, i: int):
    mx, line = np.abs(A[i, i]), i
    for j in range(i+1, A.shape[0]):
        if np.abs(A[j, i]) > mx:
            mx = np.abs(A[j, i])
            line = j
    for j in range(A.shape[0]):
        A[line, j], A[i, j] = A[i, j], A[line, j]
        L[line, j], L[i, j] = L[i, j], L[line, j]
    permutations[i], permutations[line] = permutations[line], permutations[i]

def LU_part(A: np.ndarray) -> tuple:
    L = np.empty(A.shape)
    n = A.shape[0]
    permutations = list(range(n))
    for i in range(n):
        for j in range(n):
            L[i, j] = int(i == j)
    for i in range(n):
        swap(A, L, permutations, i)
        for j in range(i+1, n):
            L[j, i] = A[j, i] / A[i, i]
            A[j] = A[j] - L[j, i] * A[i]
    return L, permutations

In [86]:
def Solve(L: np.ndarray, U: np.ndarray, b: np.ndarray, perm=[]) -> np.ndarray:
    x = np.zeros_like(b, np.float64)
    if not perm:
        perm = list(range(L.shape[0]))
    for i in range(b.shape[0]):
        x[i] = b[perm[i]]
    for i in range(1, L.shape[0]):
        for j in range(i):
            x[i]-= x[j] * L[i, j]    
    for i in range(L.shape[0]-1, -1, -1):
        for j in range(i+1, L.shape[0]):
            x[i]-= x[j] * U[i, j]
        x[i] /= U[i, i]
    return x

In [87]:
A = np.array([[-5, 1, 1, 0],
              [15, -6, 5, 1],
              [50, -16, 11, 9],
              [0, 6, 24, 44]], np.float64)
b1 = np.array([-4, -16, -2, 98], np.float64)
b2 = np.array([-61, 122, 373, -698], np.float64)
#L, U= LU_dec(A)
L, perm = LU_part(A)
x = Solve(L, A, b1, perm)
print(x)
print(A)
print(perm)

[-2. -7. -7.  7.]
[[ 50.         -16.          11.           9.        ]
 [  0.           6.          24.          44.        ]
 [  0.           0.           6.5          7.1       ]
 [  0.           0.           0.           0.38461538]]
[2, 3, 1, 0]


In [88]:
def get_matrix(n: int) -> np.ndarray:
    f = lambda i, j: np.tan(i + 1)**(17 - j)
    A = np.array([[f(i, j) for j in range(n)] for i in range(n)])
    return A

In [89]:
#Решение обычной декомпозицией
A = get_matrix(5)
b = A.dot(np.array([33 for _ in range(5)]))
L, U = LU_dec(A)
x = Solve(L, U, b)
print(x)
print(np.allclose(A.dot(x), b))

[33. 33. 33. 33. 33.]
True


In [90]:
#схема частичного выбора
L, perm = LU_part(A)
x = Solve(L, A, b, perm)
print(x)
A = get_matrix(5)
print(np.allclose(A.dot(x), b))

[33. 33. 33. 33. 33.]
True


In [91]:
#Возмущенная матрица обычной декомпозицией
A_ = get_matrix(5)
A_[0, 0] += 1e-3
b = A.dot(np.array([33 for _ in range(5)]))
L, U = LU_dec(A_)
x1 = Solve(L, U, b)
print(x1)
print(A_.dot(x1) - b)

[32.99999171 32.99996228 32.99998698 33.00006978 33.00001011]
[ 0.00000000e+00 -1.86264515e-09  1.03397577e-25 -1.13686838e-12
 -3.81469727e-06]


In [92]:
#Возмущенная матрица по схеме частичного выбора
L, perm = LU_part(A_)
x2 = Solve(L, A_, b, perm)
A_ = get_matrix(5); A_[0, 0] += 1e-3
print(x2)
print(A_.dot(x2) - b)

[32.99999171 32.99996228 32.99998698 33.00006978 33.00001011]
[ 0.00000000e+00 -3.72529030e-09 -5.16987883e-26  0.00000000e+00
  0.00000000e+00]


In [93]:
A = get_matrix(5)
nu_b = Norm(A) * Norm(np.linalg.inv(A))
d_x = Norm(x - x1) / Norm(x1)
d_A = Norm(A_ - A) / Norm(A)

In [94]:
nu_b, d_x, d_A

(1.2651682308961201e+20, 1.1430333150025495e-06, 7.177350076381855e-13)

In [95]:
d_x, nu_b * d_A

(1.1430333150025495e-06, 90805552.98658165)

In [96]:
def get_error1(n: int) -> np.float64:
    A = get_matrix(n)
    b = A.dot(np.array([33 for _ in range(n)]))
    L, U = LU_dec(A)
    x = Solve(L, U, b)
    A[0, 0] += 1e-3
    L, U = LU_dec(A)
    x_ = Solve(L, U, b)
    return Norm(x - x_)/Norm(x_)

In [97]:
def get_error2(n: int) -> np.float64:
    A = get_matrix(n)
    b = A.dot(np.array([33 for _ in range(n)]))
    L, perm = LU_part(A)
    x = Solve(L, A, b, perm)
    A_ = get_matrix(n); A_[0, 0] += 1e-3
    L, perm = LU_part(A_)
    x_ = Solve(L, A_, b, perm)
    return Norm(x - x_)/Norm(x_)

In [ ]:
fig, axs = plt.subplots()

x_data = list(range(4, 26))

axs.plot(x_data, [get_error1(i) for i in x_data], color="blue")
axs.plot(x_data, [get_error2(i) for i in x_data], color="green")
ax = plt.gca()
plt.yscale('log')
# plot X - axis    
ax.axhline(y=0, color='k')
axs.grid(True)
#fig.savefig("3.1_plot.png", dpi=500)